In [ ]:
import os
import numpy as np
import shutil
from PIL import Image
import pydicom

# Function to load and process DICOM image
def load_dicom_image(path):
    dicom_image = pydicom.dcmread(path)
    img_array = dicom_image.pixel_array

    # Normalize the pixel array values to range from 0 to 255 for display purposes
    img_array = img_array.astype(np.float32)
    img_array -= np.min(img_array)  # Shift the minimum value to 0
    img_array /= np.max(img_array)  # Scale to a range of 0 to 1
    img_array *= 255.0  # Scale to a range of 0 to 255
    img_array = img_array.astype(np.uint8)  # Convert to unsigned 8-bit integer

    # Check the rightmost pixels to see if they're black (near 0 intensity)
    rightmost_pixels = img_array[:, -10:]  # Check the last 10 pixels (right edge)

    # If the rightmost pixels are not black (intensity not close to 0), flip the image
    if np.any(rightmost_pixels > 10):  # Threshold to allow some small variance in pixel intensity
        img_array = np.flip(img_array, axis=1)  # Flip horizontally (left to right)

    img = Image.fromarray(img_array)
    return img

# Function to process all DICOM files in the folder
def process_dicom_folder(input_folder, output_folder):
    # Create the 'Aligned' folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        # Check if the file is a DICOM file (ends with .dcm)
        if filename.lower().endswith('.dcm'):
            input_path = os.path.join(input_folder, filename)
            output_path = os.path.join(output_folder, f"{os.path.splitext(filename)[0]}.tiff")  # Save as tiff

            # Process the DICOM file
            img = load_dicom_image(input_path)

            # Save the processed (flipped or original) image to the 'Aligned' folder as PNG
            img.save(output_path, format='TIFF')

    print(f"Processed {len(os.listdir(output_folder))} images. All saved to '{output_folder}'.")

# Example usage:
input_folder = 'IMG'  # Replace with the path to your DICOM files
output_folder = 'Aligned_TIFF'  # Folder where aligned images will be saved

# Process all DICOM files in the folder
process_dicom_folder(input_folder, output_folder)